In [5]:
import glob
import subprocess
import calendar

def gen_qsub(radar_start='', radar_end='', ncpu=16, walltime=10, py_script='',queue_mode=''):
    #build header
    conf_txt = \
    """#!/bin/bash
    #PBS -P kl02
    #PBS -q {qmode}
    #PBS -l walltime={time}:00:00
    #PBS -l mem={mem}GB
    #PBS -l wd
    #PBS -l ncpus={cpu}
    #PBS -l jobfs=150GB
    #PBS -m abe
    #PBS -l storage=gdata/rq0+gdata/kl02+gdata/en0+gdata/hj10+gdata/ub4
    export PATH="/g/data/en0/jss548/miniconda3:$PATH"
    source activate radar-dev
    python {python_script} -s {d1} -e {d2}
    """.format(time=walltime, cpu=ncpu, mem=int(ncpu*2), d1=radar_start, d2=radar_end, python_script=py_script, qmode=queue_mode)
    return conf_txt

#main function


#paths
qsub_path    = "/home/548/jss548/dev/tmp/qsub_cp2_level1b"
py_script    = '/home/548/jss548/dev/projects/CP-2/cp2_processing/scripts/dask_pack.py' #running daily unpack/convert/repack instead of yearly
run_qsub_cmd = "qsub -k oe"
ncpu         = 16
walltime     = 24
start_year   = 2014
end_year     = 2014
month_list   = [[11,11]]#[[1,1],[2,2],[3,3],[4,4],[5,5],[6,6],[7,7],[8,8],[9,9],[10,10],[11,11],[12,12]]
run_flag     = True
qmode        = 'normal'

count = 0
for year in range(start_year, end_year+1):        
    for i, month_set in enumerate(month_list):

        #generate qsub for wuater sets
        month_end_day = calendar.monthrange(year, month_set[1])[1]
        start_date = str(year) + str(month_set[0]).zfill(2) + '01'
        end_date   = str(year) + str(month_set[1]).zfill(2) + str(month_end_day)

        f     = gen_qsub(start_date, end_date, ncpu, walltime, py_script, qmode)
        fname = '/'.join([qsub_path, "cp2_l1b_%04i_m%i_m%i.pbs" % (year, month_set[0], month_set[1])])
        print(fname)
        with open(fname, 'w') as fid:
            fid.write(f)
        count += 1
        if run_flag == True:
            cmd           = ' '.join([run_qsub_cmd,fname])
            process       = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
            output, error = process.communicate()
            print('runing ' + fname)
            print(output)

print('finished qsub generate for ',str(count), 'jobs')

/home/548/jss548/dev/tmp/qsub_cp2_level1b/cp2_l1b_2014_m11_m11.pbs
runing /home/548/jss548/dev/tmp/qsub_cp2_level1b/cp2_l1b_2014_m11_m11.pbs
b'14589734.gadi-pbs\n'
finished qsub generate for  1 jobs
